In [1]:
import xml.etree.ElementTree as ET
xml_file_path = "copa-test.xml"


In [2]:
tree = ET.parse(xml_file_path)
root = tree.getroot()

In [33]:
import numpy as np
import pandas as pd

num = len(root)
items_id = list(np.zeros(num))
asks_for = list(np.zeros(num))
p_contents = list(np.zeros(num))
a1_contents = list(np.zeros(num))
a2_contents = list(np.zeros(num))
correct_answers = list(np.zeros(num))

In [34]:
root[0].find("p").text

'The item was packaged in bubble wrap.'

In [35]:
for i, child in enumerate(root):
    items_id[i] = child.attrib['id']
    asks_for[i] = child.attrib['asks-for']
    p_contents[i] = child.find("p").text
    a1_contents[i] = child.find("a1").text
    a2_contents[i] = child.find("a2").text
    correct_answers[i] = child.attrib['most-plausible-alternative']

In [51]:
from googletrans import Translator

def translate_to_kazakh(text):
    translator = Translator()
    translated = translator.translate(text, dest='kk')  # 'kk' is the language code for Kazakh
    return translated.text

# Example usage:
english_text = p_contents[0]
kazakh_translation = translate_to_kazakh(english_text)
print("English:", english_text)
print("Kazakh Translation:", kazakh_translation)

English: The item was packaged in bubble wrap.
Kazakh Translation: Элемент көпіршік орамаларына оралды.


In [46]:
from tqdm import tqdm

def translation_process(texts):
    english_texts = texts
    kazakh_translations = []

    with tqdm(total=len(english_texts), desc="Translating") as pbar:
        for text in english_texts:
            translated_text = translate_to_kazakh(text)
            kazakh_translations.append(translated_text)
            pbar.update(1)
    
    return kazakh_translations

In [47]:
p_contents_kz = translation_process(p_contents)

Translating: 100%|██████████| 500/500 [04:51<00:00,  1.72it/s]


In [48]:
a1_contents_kz = translation_process(a1_contents)

Translating: 100%|██████████| 500/500 [04:40<00:00,  1.78it/s]


In [50]:
a2_contents_kz = translation_process(a2_contents)

Translating: 100%|██████████| 500/500 [04:44<00:00,  1.76it/s]


In [52]:
data = {
    'items_id' : items_id,
    'asks-for': asks_for,
    'p_contents_eng' : p_contents,
    'a1_contents_eng' : a1_contents,
    'a2_contents_eng' : a2_contents,
    'correct_answers' : correct_answers,
    'p_contents_kz' : p_contents_kz,
    'a1_contents_kz' : a1_contents_kz,
    'a2_contents_kz' : a2_contents_kz
}

df = pd.DataFrame(data)

In [53]:
df

,items_id,asks-for,p_contents_eng,a1_contents_eng,a2_contents_eng,correct_answers,p_contents_kz,a1_contents_kz,a2_contents_kz
0,501,cause,The item was packaged in bubble wrap.,It was fragile.,It was small.,1,Элемент көпіршік орамаларына оралды.,Бұл нәзік болды.,Бұл кішкентай болды.
1,502,effect,I emptied my pockets.,I retrieved a ticket stub.,I found a weapon.,1,Мен қалтамды босатамын.,Мен билет кестесін шығардым.,Мен қару таптым.
2,503,effect,Termites invaded the house.,The termites disappeared from the house.,The termites ate through the wood in the house.,2,Термиттер үйді басып алды.,Термиттер үйден жоғалып кетті.,Терметтер үйдегі ағаштан жеді.
3,504,effect,The travelers reached the border.,The patrol agent checked their passports.,The patrol agent accused them of smuggling.,1,Саяхатшылар шекараға жетті.,Патрульдік агент паспорттарын тексерді.,Патрульдік агент оларды контрабанданың деп айы...
4,505,cause,The office was closed.,It was a holiday.,It was summer.,1,Кеңсе жабылды.,Бұл мереке болды.,Бұл жаз болды.
...,...,...,...,...,...,...,...,...,...
495,996,effect,The runner sensed his competitor gaining on him.,He dropped out of the race.,He sped up his pace.,2,Жүгіруші өзінің бәсекелесін өзіне алғанын сезді.,Ол жарыстан кетті.,Ол өзінің қарқынын көтерді.
496,997,effect,I thought carefully about the problem.,I asked for advice.,I came up with a solution.,2,Мен проблеманы мұқият ойладым.,Мен кеңес сұрадым.,Мен шешім қабылдадым.
497,998,effect,The traveler walked on the shaky suspension br...,He felt terrified.,He felt ecstatic.,1,Саяхатшы Шаки аспалы көпірінде жүрді.,Ол қатты қорасты.,Ол экстаздық сезінді.
498,999,effect,The man anticipated the team's victory.,He met his friends to watch the game.,He made a bet with his friends.,2,Ер адам команданың жеңісін болжады.,Ол достарымен кездесті.,Ол достарымен ставка жасады.


In [54]:
df.to_csv('copa-translated.csv', index=False)